In [119]:
import numpy as np
import pandas as pd
import os

from drugsdataset import DrugsDataset
from sklearn.model_selection import KFold
import xgboost as xgb

# GLOBAL OPTIONS
NUM_FOLDS = 5
SEED = 2137

# model we wish to validate
model = xgb.XGBClassifier(objective ='multi:softprob',
                        colsample_bytree = 0.3,
                        learning_rate = 0.1,
                        max_depth = 15,
                        alpha = 10,
                        n_estimators = 25,
                        verbosity=1)

In [2]:
drugsdataset = DrugsDataset("../data/")

In [6]:
bow_train, bow_test = drugsdataset.create_bag_of_words()

In [79]:
X_train = bow_train
y_train = drugsdataset.train['rate'].astype(float)

In [31]:
kf = KFold(n_splits = NUM_FOLDS, random_state = SEED, shuffle = True)

In [120]:
predictions = np.array([0.0 for i in range(len(y_train))])

for train_index, test_index in kf.split(X_train):
    X_tr, X_valid = X_train[train_index], X_train[test_index]
    y_tr, y_valid = y_train[train_index], y_train[test_index]
    

    model.fit(X_tr, y_tr)
    predictions[test_index] = model.predict(X_valid)
    print(predictions[test_index])

[10. 10. 10. ... 10. 10. 10.]
[10. 10. 10. ... 10. 10. 10.]
[10. 10. 10. ... 10. 10.  1.]
[ 1. 10. 10. ... 10. 10. 10.]
[ 9.  1. 10. ... 10. 10. 10.]


In [121]:
# Accuracy
np.sum(predictions==y_train)/len(y_train)

0.38058

In [122]:
# Mean absolute error
np.sum(np.abs(predictions - y_train)) / len(y_train)

2.3938866666666665

In [126]:
for i in range(10):
    print(np.sum(predictions == (i+1)) / len(predictions), end="-")
    print(np.sum(y_train == (i+1)) / len(y_train), end="-")
    print("")

0.12546-0.13437333333333334-
0.00018666666666666666-0.04287333333333333-
0.0005333333333333334-0.04037333333333333-
4e-05-0.03104-
0.00044666666666666666-0.04975333333333333-
1.3333333333333333e-05-0.03922-
0.00021333333333333333-0.058673333333333334-
0.009693333333333333-0.11702666666666667-
0.04737333333333334-0.17018-
0.81604-0.3164866666666667-
